In [64]:
# Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import dask.dataframe as dd
import gc


In [3]:
import pandas as pd

chunk_size = 10000  # Adjust based on memory availability
dfs = []  # List to store processed chunks

for chunk in pd.read_csv('GSE139495_Single_cell_Raw_counts_10252019.tsv', sep="\t", index_col=0, chunksize=chunk_size):
    # Process each chunk (modify as needed)
    chunk = chunk.astype('float32')  # Example: Convert to smaller data type
    dfs.append(chunk)  # Store processed chunk

# Combine chunks into a single DataFrame
df2 = pd.concat(dfs)




In [4]:
# Display info
df2.info()
df2.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 33538 entries, MIR1302-2HG to FAM231C
Columns: 12341 entries, ACAGCTAAGGGTTCCC to TTTACTGCATCCCACT
dtypes: float32(12341)
memory usage: 1.5+ GB


,ACAGCTAAGGGTTCCC,ACATACGTCATTTGGG,ACATCAGAGTCGTACT,AGAGTGGGTGCAACTT,AGGCCACGTTCGCGAC,AGTGGGACATCCGGGT,ATCACGAGTCCAGTTA,ATCATGGGTCCGACGT,ATTACTCCAGATCTGT,CAAGATCGTCCATCCT,...,TCGGGACAGGACTGGT,TCTATTGAGCCTCGTG,TCTTTCCAGTCTTGCA,TGCCAAAAGCGTCAAG,TGCGGGTTCCTCATTA,TGCTACCTCTCGCATC,TGGCTGGTCGGTGTTA,TTAGGACTCCACGACG,TTGGAACGTCCCTTGT,TTTACTGCATCCCACT
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL732372.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AC114498.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33538 entries, MIR1302-2HG to FAM231C
Columns: 12341 entries, ACAGCTAAGGGTTCCC to TTTACTGCATCCCACT
dtypes: float32(12341)
memory usage: 1.5+ GB


In [21]:
df2.dtypes

ACAGCTAAGGGTTCCC    float32
ACATACGTCATTTGGG    float32
ACATCAGAGTCGTACT    float32
AGAGTGGGTGCAACTT    float32
AGGCCACGTTCGCGAC    float32
                     ...   
TGCTACCTCTCGCATC    float32
TGGCTGGTCGGTGTTA    float32
TTAGGACTCCACGACG    float32
TTGGAACGTCCCTTGT    float32
TTTACTGCATCCCACT    float32
Length: 12341, dtype: object

In [11]:
import pandas as pd

# Define a function to optimize the data types
def optimize_dtypes(df):
    optimized_dtypes = {}
    
    for column in df.columns:
        if df[column].dtype == 'float64':  # If the column is float64, use float32 to save memory
            optimized_dtypes[column] = 'float32'
        elif df[column].dtype == 'int64':  # If the column is int64, use int32 to save memory
            optimized_dtypes[column] = 'int32'
        elif df[column].dtype == 'object':  # If the column is object (strings), we leave it as is or convert to category
            optimized_dtypes[column] = 'category'  # Converting string columns to 'category' saves memory
            
    return optimized_dtypes

# Read the dataset with optimized dtypes
optimized_dtypes = optimize_dtypes(pd.read_parquet("C:/Users/fjrui/OneDrive/Documents/Desktop/Bootcamp/Project 4/Team_BioByte/Capstone/data.parquet"))
df2_clean = pd.read_parquet("C:/Users/fjrui/OneDrive/Documents/Desktop/Bootcamp/Project 4/Team_BioByte/Capstone/data.parquet")

# Display the first few rows to confirm
df2_clean.head()


,ACAGCTAAGGGTTCCC,ACATACGTCATTTGGG,ACATCAGAGTCGTACT,AGAGTGGGTGCAACTT,AGGCCACGTTCGCGAC,AGTGGGACATCCGGGT,ATCACGAGTCCAGTTA,ATCATGGGTCCGACGT,ATTACTCCAGATCTGT,CAAGATCGTCCATCCT,...,TCGGGACAGGACTGGT,TCTATTGAGCCTCGTG,TCTTTCCAGTCTTGCA,TGCCAAAAGCGTCAAG,TGCGGGTTCCTCATTA,TGCTACCTCTCGCATC,TGGCTGGTCGGTGTTA,TTAGGACTCCACGACG,TTGGAACGTCCCTTGT,TTTACTGCATCCCACT
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
# # Define the file path
file_path = r"C:\Users\fjrui\OneDrive\Documents\Desktop\Bootcamp\Project 4\Team_BioByte\Capstone\data.parquet"  # Update with the correct path if needed

# # Load the TSV file into a DataFrame
# df2 = pd.read_csv(file_path, sep="\t", index_col=0)  # Use index_col=0 if the first column contains gene names

# # Display basic info and the first few rows
# #df2.head(10)  # Show first few rows

In [17]:
df2.to_parquet("data.parquet", compression="snappy")


In [25]:
# import pandas as pd

# # Load the Parquet file
# df2 = pd.read_parquet("C:/Users/fjrui/OneDrive/Documents/Desktop/Bootcamp/Project 4/Team_BioByte/Capstone/data.parquet")

# # Display the first few rows
# df2.head(20)


In [27]:
len(df2)

33538

In [29]:
df2.columns

Index(['ACAGCTAAGGGTTCCC', 'ACATACGTCATTTGGG', 'ACATCAGAGTCGTACT',
       'AGAGTGGGTGCAACTT', 'AGGCCACGTTCGCGAC', 'AGTGGGACATCCGGGT',
       'ATCACGAGTCCAGTTA', 'ATCATGGGTCCGACGT', 'ATTACTCCAGATCTGT',
       'CAAGATCGTCCATCCT',
       ...
       'TCGGGACAGGACTGGT', 'TCTATTGAGCCTCGTG', 'TCTTTCCAGTCTTGCA',
       'TGCCAAAAGCGTCAAG', 'TGCGGGTTCCTCATTA', 'TGCTACCTCTCGCATC',
       'TGGCTGGTCGGTGTTA', 'TTAGGACTCCACGACG', 'TTGGAACGTCCCTTGT',
       'TTTACTGCATCCCACT'],
      dtype='object', length=12341)

In [31]:
index_values = df2.index
index_values

Index(['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3',
       'AL627309.2', 'AL627309.4', 'AL732372.1', 'OR4F29', 'AC114498.1',
       ...
       'AC007325.2', 'BX072566.1', 'AL354822.1', 'AC023491.2', 'AC004556.1',
       'AC233755.2', 'AC233755.1', 'AC240274.1', 'AC213203.1', 'FAM231C'],
      dtype='object', length=33538)

In [33]:
index_list = df2.index.tolist()

# Print only the first 10 values
print(index_list[:10])

['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3', 'AL627309.2', 'AL627309.4', 'AL732372.1', 'OR4F29', 'AC114498.1']


In [35]:
# Sample DataFrame
df2_clean = pd.DataFrame(df2)

# Create a new column "Gen" with the index values
df2_clean['Gen'] = df2.index

# Reset the index to create a new index column
df2_clean = df2.reset_index(drop=False)

df2_clean

,index,ACAGCTAAGGGTTCCC,ACATACGTCATTTGGG,ACATCAGAGTCGTACT,AGAGTGGGTGCAACTT,AGGCCACGTTCGCGAC,AGTGGGACATCCGGGT,ATCACGAGTCCAGTTA,ATCATGGGTCCGACGT,ATTACTCCAGATCTGT,...,TCGGGACAGGACTGGT,TCTATTGAGCCTCGTG,TCTTTCCAGTCTTGCA,TGCCAAAAGCGTCAAG,TGCGGGTTCCTCATTA,TGCTACCTCTCGCATC,TGGCTGGTCGGTGTTA,TTAGGACTCCACGACG,TTGGAACGTCCCTTGT,TTTACTGCATCCCACT
0,MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AL627309.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,AC233755.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33534,AC233755.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33535,AC240274.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33536,AC213203.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
df2_clean["index"].value_counts()

index
MIR1302-2HG    1
DCT            1
GPC6-AS2       1
GPC6           1
AL354811.1     1
              ..
ORC3           1
RARS2          1
SLC35A1        1
AL049697.1     1
FAM231C        1
Name: count, Length: 33538, dtype: int64

In [40]:
cell_counts = df2_clean["index"].value_counts().sort_values(ascending=True)

cell_counts

index
MIR1302-2HG    1
AL627309.2     1
AL627309.4     1
AL732372.1     1
OR4F29         1
              ..
AC213203.1     1
MT-ND6         1
DNMT3L         1
MT-ND4L        1
FAM231C        1
Name: count, Length: 33538, dtype: int64

In [36]:
# Assuming 'df' is your DataFrame and the column you're interested in is named 'patient'
value_counts = df2_clean['index'].value_counts()

# Filter for values that have only one occurrence
values_with_one_count = value_counts[value_counts == 1]

# Print or return the values that occur exactly once
values_with_one_count


index
MIR1302-2HG    1
DCT            1
GPC6-AS2       1
GPC6           1
AL354811.1     1
              ..
ORC3           1
RARS2          1
SLC35A1        1
AL049697.1     1
FAM231C        1
Name: count, Length: 33538, dtype: int64

In [42]:
len(df2_clean)

33538

In [44]:
# # Assuming 'df2_clean' is your original DataFrame
# new_df = df2_clean[df2_clean.index == 0]

# # Display the new dataset
# new_df


In [44]:
# Assuming 'df' is your DataFrame
# First, let's filter the rows where the index value is equal to 1
filtered_df = df2_clean[df2_clean.index == 1]

# Reshape the data into a new DataFrame where the columns contain their respective values
new_dataset = filtered_df.melt(var_name="Column", value_name="Value")

# Print the new dataset
new_dataset


,Column,Value
0,index,FAM138A
1,ACAGCTAAGGGTTCCC,0.0
2,ACATACGTCATTTGGG,0.0
3,ACATCAGAGTCGTACT,0.0
4,AGAGTGGGTGCAACTT,0.0
...,...,...
12337,TGCTACCTCTCGCATC,0.0
12338,TGGCTGGTCGGTGTTA,0.0
12339,TTAGGACTCCACGACG,0.0
12340,TTGGAACGTCCCTTGT,0.0


In [46]:
print(df2_clean.columns)


Index(['index', 'ACAGCTAAGGGTTCCC', 'ACATACGTCATTTGGG', 'ACATCAGAGTCGTACT',
       'AGAGTGGGTGCAACTT', 'AGGCCACGTTCGCGAC', 'AGTGGGACATCCGGGT',
       'ATCACGAGTCCAGTTA', 'ATCATGGGTCCGACGT', 'ATTACTCCAGATCTGT',
       ...
       'TCGGGACAGGACTGGT', 'TCTATTGAGCCTCGTG', 'TCTTTCCAGTCTTGCA',
       'TGCCAAAAGCGTCAAG', 'TGCGGGTTCCTCATTA', 'TGCTACCTCTCGCATC',
       'TGGCTGGTCGGTGTTA', 'TTAGGACTCCACGACG', 'TTGGAACGTCCCTTGT',
       'TTTACTGCATCCCACT'],
      dtype='object', length=12342)


In [50]:
df_transposed = df2_clean.set_index('index').T.astype(int)
df_transposed

index,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.4,AL732372.1,OR4F29,AC114498.1,...,AC007325.2,BX072566.1,AL354822.1,AC023491.2,AC004556.1,AC233755.2,AC233755.1,AC240274.1,AC213203.1,FAM231C
ACAGCTAAGGGTTCCC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACATACGTCATTTGGG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACATCAGAGTCGTACT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AGAGTGGGTGCAACTT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AGGCCACGTTCGCGAC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGCTACCTCTCGCATC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TGGCTGGTCGGTGTTA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTAGGACTCCACGACG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGGAACGTCCCTTGT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_transposed.head(10)

In [ ]:
import pandas as pd
import pyarrow.parquet as pq

# Function to process the data in chunks
def find_values_equals_one_parquet(file_path, chunk_size=50000):
    # Load the parquet file using pandas
    df = pd.read_parquet(file_path)
    
    # Iterate over the dataframe in chunks
    num_chunks = len(df) // chunk_size  # Determine how many chunks we need
    for chunk_idx in range(num_chunks + 1):  # Include the last partial chunk
        chunk = df[chunk_idx * chunk_size : (chunk_idx + 1) * chunk_size]
        
        # Iterate over each column to find where the values are equal to 1
        for column in chunk.columns:
            mask = chunk[column] == 1  # Apply condition column by column
            
            # Extract rows where the value is 1
            result = chunk[mask]
            
            # If any rows match, yield the index and column name
            if not result.empty:
                yield result.index, column

# Specify your Parquet file path
file_path = r"C:\Users\fjrui\OneDrive\Documents\Desktop\Bootcamp\Project 4\Team_BioByte\Capstone\data.parquet"

# Use the function to process and find 1's in the dataset
for index, column in find_values_equals_one_parquet(file_path):
    print(f"Found value 1 at index: {index} in column: {column}")


Found value 1 at index: Index(['CLEC4E', 'NUDT16L1', 'NTAN1'], dtype='object') in column: ACAGCTAAGGGTTCCC
Found value 1 at index: Index(['PDIA6', 'RPLP2', 'PXMP2', 'ACTG1', 'C5AR2', 'IGLC2', 'MT-ATP6'], dtype='object') in column: ACATACGTCATTTGGG
Found value 1 at index: Index(['PIK3CG', 'CCNB1IP1', 'AC022087.1'], dtype='object') in column: ACATCAGAGTCGTACT
Found value 1 at index: Index(['MT-ND2'], dtype='object') in column: AGAGTGGGTGCAACTT
Found value 1 at index: Index(['HBB'], dtype='object') in column: AGGCCACGTTCGCGAC
Found value 1 at index: Index(['ACTB', 'FTH1'], dtype='object') in column: AGTGGGACATCCGGGT
Found value 1 at index: Index(['S100A8', 'FCGR3B', 'THAP4', 'RPS6', 'MALAT1', 'RPL19', 'RPL13A',
       'IGLC2'],
      dtype='object') in column: ATCACGAGTCCAGTTA
Found value 1 at index: Index(['THG1L', 'HLA-DRA', 'VPS50', 'TPT1', 'RPLP1', 'FAM96B', 'UBA52'], dtype='object') in column: ATCATGGGTCCGACGT
Found value 1 at index: Index(['RPL11', 'CD38'], dtype='object') in column